# 此次训练配置
填写或修改后再运行此ipynb

In [ ]:
session_name = '3D_icon'

## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    print(output)
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False
if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
        # f.write("")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")

In [ ]:
new_channels = [
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/peterjc123/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/"
]

current_channels = !conda config --show channels
current_channels = ('').join(current_channels).split()

# print(current_channels)

for channel in new_channels:
    if channel in current_channels:
        print(f"此channel已经在列表中：{channel}")
    else:
        !conda config --add channels $channel
        
!conda config --set show_channel_urls yes

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
# from func.env import detect_environment
# env_cb = detect_environment()
# env_name = env_cb['env_name']
# content_path = env_cb['content_path']
# print(root_dir)

%cd $repo_root_dir
# 装载nbfunc中的hub涉及的所有ipynb函数
%run ./nbfunc/hub.ipynb

initGit()

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
]

tools_to_install = [
    "iputils-ping",
    'lsof'
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install --user $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
# from func.env import setProxy
# try:
#     proxy,region
# except NameError:
#     sp = setProxy()
#     proxy = sp['proxy']
#     region = sp['region']
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb
selProxy(True,'自供')
print(proxy)
print(proxyURL)

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

python_version = '3.10.6'

# if env_name == 'AutoDL':
custom_env_name = 'py' + python_version
# elif env_name == 'OpenBayes':
#     custom_env_name = 'base'

s = getoutput('conda info --env')
print(s)
if custom_env_name not in s:
    print(f'当前系统中没有{custom_env_name}环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境
    # 更新bashrc中的环境变量
    # 在 /openbayes/home 下创建一个新的环境：https://openbayes.com/docs/gear/conda/#%E4%BD%BF%E7%94%A8-conda-%E5%88%9B%E5%BB%BA%E6%96%B0%E7%9A%84%E7%8E%AF%E5%A2%83
    python_install_args = [
        'source /usr/local/etc/profile.d/conda.sh','&&',
        f'cd {root_dir}','&&',
        f'conda create -p {custom_env_name} python={python_version} -y','&&',
        'conda init bash','&&',
        'source /root/.bashrc -y'
    ]
    
    # !source /usr/local/etc/profile.d/conda.sh &&\
    # conda create -n $custom_env_name python=$python_version -y &&\
    # conda init bash && source /root/.bashrc -y
    
    python_install_cmd = (" ").join(python_install_args)
    !$python_install_cmd
    
else:
    print(f'已经安装了{custom_env_name}环境')

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## 交代Base_args

In [ ]:
from func.env import setProxy,getWebUIDir
import subprocess

def find_conda_env(name):
    # Get a list of all conda environments on the system
    output = subprocess.run(["conda", "env", "list"], capture_output=True)
    envs = output.stdout.decode().split("\n")

    # Find the path of the environment
    for env in envs:
        if name in env:
            path = env.split()[-1]
            print(f"The path of the environment '{name}' is {path}")
            return path
    else:
        print(f"Environment '{name}' not found.")
        return None



try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

webUIDir=getWebUIDir()
print(webUIDir)

custom_env_dir = find_conda_env(custom_env_name)

base_args = [
    'source activate','&&',
    f'conda activate {custom_env_dir}','&&',
    proxy,'&&'
       ]

In [ ]:
print(base_args)

## 创建训练Session

In [ ]:
import os

# 相对固定的配置
training_sessions_name = 'training_sessions'

# 创建training_sessions文件夹
trainning_sessions_dir = os.path.join(root_dir, training_sessions_name)
if not os.path.exists(trainning_sessions_dir):
    os.mkdir(trainning_sessions_dir)

# 创建session_name文件夹
project_dir = os.path.join(trainning_sessions_dir, session_name)
if not os.path.exists(project_dir):
    os.mkdir(project_dir)

# 在project_name目录下创建子目录
subdirs = [
    'model', # 模型输出
    'img', # 训练集
    'reg' , # 正则化图集
    'config' # 配置
]

for subdir in subdirs:
    subdir_path = os.path.join(project_dir, subdir)
    if not os.path.exists(subdir_path):
        os.mkdir(subdir_path)


# 创建标准配置表

In [ ]:
import json,os

standard_json = {
    "pretrained_model_name_or_path": "",
    "v2": False,
    "v_parameterization": False,
    "logging_dir": "",
    "train_data_dir": "",
    "reg_data_dir": "",
    "output_dir": "",
    "max_resolution": "512,512",
    "learning_rate": "6e-5",
    "lr_scheduler": "cosine_with_restarts",
    "lr_warmup": "0",
    "train_batch_size": 2,
    "epoch": 1,
    "save_every_n_epochs": 1,
    "mixed_precision": "bf16",
    "save_precision": "bf16",
    "seed": "1234",
    "num_cpu_threads_per_process": 2,
    "cache_latents": True,
    "caption_extension": ".txt",
    "enable_bucket": False,
    "gradient_checkpointing": False,
    "full_fp16": False,
    "no_token_padding": False,
    "stop_text_encoder_training": 0,
    "xformers": True,
    "save_model_as": "safetensors",
    "shuffle_caption": False,
    "save_state": False,
    "resume": "",
    "prior_loss_weight": 1.0,
    "text_encoder_lr": "7e-6",
    "unet_lr": "6e-5",
    "network_dim": 128,
    "lora_network_weights": "",
    "color_aug": False,
    "flip_aug": False,
    "clip_skip": 1,
    "gradient_accumulation_steps": 1.0,
    "mem_eff_attn": False,
    "output_name": "",
    "model_list": "custom",
    "max_token_length": "75",
    "max_train_epochs": "20",
    "max_data_loader_n_workers": "1",
    "network_alpha": 64,
    "training_comment": "",
    "keep_tokens": "0",
    "lr_scheduler_num_cycles": "",
    "lr_scheduler_power": "",
    "persistent_data_loader_workers": True,
    "bucket_no_upscale": True,
    "random_crop": False,
    "bucket_reso_steps": 64.0,
    "caption_dropout_every_n_epochs": 0.0,
    "caption_dropout_rate": 0,
    "optimizer": "Lion",
    "optimizer_args": "",
    "noise_offset": "0.1",
    "LoRA_type": "Standard",
    "conv_dim": 1,
    "conv_alpha": 1,
    "sample_every_n_steps": 0,
    "sample_every_n_epochs": 0,
    "sample_sampler": "euler_a",
    "sample_prompts": "",
    "additional_parameters": ""
}

import datetime

def generate_date():
    today = datetime.datetime.today()
    date_str = today.strftime("%m%d")
    return date_str

root_dir = '/openbayes/home'

path_dict = {
    'pretrained_model_name_or_path': f'{root_dir}/base_model/chilloutmix_NiPrunedFp16.safetensors',
    'logging_dir': f'{root_dir}/tf_dir',
    'train_data_dir': f'{root_dir}/training_sessions/{session_name}/img',
    'reg_data_dir': f'{root_dir}/training_sessions/{session_name}/reg',
    'output_dir': f'{root_dir}/training_sessions/{session_name}/model',
    'output_name': f'{session_name}_v1_{generate_date()}_',
}

standard_json.update(path_dict)

def save_json_to_file(data, file_path):
    # 获取保存目录的父目录
    parent_dir = os.path.dirname(file_path)
    
    # 如果目录不存在，就创建目录
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    # 如果文件不存在，就写入数据到指定文件路径
    if not os.path.isfile(file_path):
        with open(file_path, 'w') as f:
            json.dump(data, f)

config_path = os.path.join(root_dir, training_sessions_name, session_name, 'config', 'default.json')
save_json_to_file(standard_json, config_path)

In [ ]:
# import json

# def read_and_print_json(file_path):
#     with open(file_path) as f:
#         data = json.load(f)
#         print(json.dumps(data, indent=4))

# file_path = "/openbayes/home/training_sessions/LoRA/config/0318_SDL_ThaiZhu_0.4_Lion_Deepbooru_Dataset_10.json"
# read_and_print_json(file_path)


## <i class="fa fa-bolt"></i> 安装Xformers和triton

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

### 从源代码的方式安装triton

In [ ]:
# triton_repo_dir = os.path.join(root_dir,'triton')

# if not os.path.exists(triton_repo_dir):
#     !cd $root_dir &&\
#     git clone https://github.com/openai/triton.git

# !cd $root_dir &&\
# cd triton/python &&\
# pip install --user cmake &&\
# pip -v install -e .

### 通过PIP安装triton的最新nightly release

In [ ]:
triton_install_args = base_args[:]
triton_install_args += [
    'python -m pip install --user -U --pre triton'
]
triton_install_cmd = (" ").join(triton_install_args)
print(triton_install_cmd)
!$triton_install_cmd

### 从源代码的方式安装Xformers

In [ ]:
import subprocess,os,time
'''
执行!python -m xformers.info命令会返回类似以下格式的信息，请参考

xFormers 0.0.16+bc08bbc.d20230122
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.flshattF:               available
memory_efficient_attention.flshattB:               available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.tritonflashattF:        available
memory_efficient_attention.tritonflashattB:        available
swiglu.fused.p.cpp:                                available
is_triton_available:                               True
is_functorch_available:                            False
pytorch.version:                                   1.12.1+cu113
pytorch.cuda:                                      available
gpu.compute_capability:                            8.6
gpu.name:                                          NVIDIA GeForce RTX 3090
build.info:                                        available
build.cuda_version:                                1103
build.python_version:                              3.8.13
build.torch_version:                               1.12.1+cu113
build.env.TORCH_CUDA_ARCH_LIST:                    None
build.env.XFORMERS_BUILD_TYPE:                     None
build.env.XFORMERS_ENABLE_DEBUG_ASSERTIONS:        None
build.env.NVCC_FLAGS:                              None
build.env.XFORMERS_PACKAGE_FROM:                   None
source.privacy:                                    open source
'''

def get_python_version_and_parse():
    # Get the current python version
    version = subprocess.run(['python', '-V'], stdout=subprocess.PIPE).stdout.decode().strip()
    version = version.split(' ')[1]
    major, minor, patch = version.split('.')
    # formatted_version = f'cp{major}{minor[0]}{patch[0]}'
    formatted_version = f'cp{major}{minor[0]}'
    return formatted_version


def get_xformers_info():
    get_xformers_info_args = base_args[:]
    get_xformers_info_args += [
        f'cd {webUIDir}','&&',
        'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m xformers.info'
    ]
    get_xformers_info_cmd = (" ").join(get_xformers_info_args)
    print(get_xformers_info_cmd)
    output = !$get_xformers_info_cmd
    output = '\n'.join(output)
    info = {}
    for line in output.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            info[key.strip()] = value.strip()
    return info

def check_xformers_info(validate_rules):
    info = get_xformers_info()
    print((" ").join(info))
    for key, value in validate_rules.items():
        if info.get(key) != value:
            if isinstance(value, bool):
                if str(value) != info.get(key):
                    return False
            else:
                return False
    return True

def find_xformers_whl_and_install(root):
    py_short_ver = get_python_version_and_parse()
    whl_name = f'xformers*{py_short_ver}*.whl'
    whl_path_list = !find $root -name $whl_name
    whl_path_list = list(filter(lambda x: x.endswith('.whl'), whl_path_list))
    print(whl_path_list)
    if whl_path_list:
        
        whl_path = whl_path_list[0]
        # subprocess.run(['pip', 'install', whl_path], stdout=subprocess.PIPE)
        !pip install --user $whl_path
        return whl_path
    else:
        return None

validate_rules = {
    'is_triton_available': True,
    'pytorch.cuda': 'available',
}

# 首先判断本机是否已经安装了可以运行的Xformers

if check_xformers_info(validate_rules):
    print('Xformers可以正常运行，无需另外安装或编译')
else:
    # 寻找Xformers预编译wheel，并安装返回列表中的第一个wheel
    for i in range(0, 5):  # 在input中进行搜索
        whl_path = find_xformers_whl_and_install(f'/input{i}')
        if whl_path:
            break
        else:
            continue
    

## <i class="fa fa-download"></i> 更新或安装Kohya本体与环境

In [ ]:
import os
import ipywidgets as widgets
import re
from IPython.display import display, clear_output

def get_project_name(project_url):
    project_name = os.path.basename(project_url)
    if project_name.endswith('.git'):
        project_name = project_name[:-4]
    return project_name

def clone_or_update_github_project(project_url, root_dir, base_args, debug=False):
    project_name = get_project_name(project_url)
    project_dir = os.path.join(root_dir, project_name)
    alertBtn = widgets.Button(
        value=False,
        description='',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
    display(alertBtn)
    if os.path.exists(project_dir):
        alertBtn.description='正在更新...'
        alertBtn.button_style='warning'
        alertBtn.icon='upload'
        clone_or_update_args = base_args[:]
        clone_or_update_args += [
            f'cd {project_dir}','&&',
            'git pull'
        ]
        clone_or_update_cmd = (" ").join(clone_or_update_args)
        print(clone_or_update_cmd)
        !$clone_or_update_cmd
        if debug == False:
            # clear_output()
            display(alertBtn)
        alertBtn.description='更新完成!'
        alertBtn.button_style='success'
        alertBtn.icon='check'
    else:
        alertBtn.description=f'安装{project_name}...'
        alertBtn.button_style='info'
        alertBtn.icon='download'
        clone_or_update_args = base_args[:]
        clone_or_update_args += [
            f'cd {root_dir}','&&',
            f'git clone {project_url}',
            '&&',
            f'cd {project_name}'
        ]
        clone_or_update_cmd = (" ").join(clone_or_update_args)
        print(clone_or_update_cmd)
        !$clone_or_update_cmd
        if debug == False:
            # clear_output()
            display(alertBtn)
        alertBtn.description=f'{project_name}安装完成!'
        alertBtn.button_style='success'
        alertBtn.icon='check'


clone_or_update_github_project(
    project_url = 'https://github.com/bmaltais/kohya_ss.git',
    root_dir = root_dir,
    base_args = base_args,
    debug = False
)

## 自动填写Config文件路径
修改Gradio文本框中的默认值

In [ ]:
import re
import os

def add_input_to_placeholder(filename, input):
    with open(filename, 'r') as f:
        content = f.read()
    
    # 使用正则表达式匹配placeholder
    placeholder_pattern = r'placeholder="type the configuration file path or use the \'Open\' button above to select it\.\.\."'
    placeholder_match = re.search(placeholder_pattern, content)

    if placeholder_match:
        # 在placeholder后面添加value
        placeholder_end = placeholder_match.end()
        new_content = content[:placeholder_end] + f'\n,value="{input}"' + content[placeholder_end:]
        
        if f',value="{input}"' not in content:
            with open(filename, 'w') as f:
                f.write(new_content)
                
            print(f"\033[33m文件{filename}已添加input值{input}\033[0m")
        else:
            print(f"\033[33m文件{filename}已包含input值{input}\033[0m")
        
    else:
        print(f"在文件{filename}中未找到placeholder")


def validate_text_in_file(file_path, expected_text):
    with open(file_path, 'r') as file:
        # 读取文件内容
        file_contents = file.read()

    # 检查是否存在所需的文本
    if expected_text in file_contents:
        # 打印成功消息
        print(f"\033[32m✅ 文件 {file_path} 包含所期望的文本 '{expected_text}'\033[0m")
    else:
        # 打印错误消息
        print(f"\033[31m❌ 文件 {file_path} 不包含所期望的文本 '{expected_text}'\033[0m")

file_path = os.path.join(root_dir, "kohya_ss/library/common_gui.py")
add_input_to_placeholder(file_path, config_path)
expected_text = config_path
validate_text_in_file(file_path, expected_text)

## 安装kohya_ss所需的python环境

In [ ]:
import os
import subprocess

def install_repo_env(project_url, root_dir, base_args):
    project_name = get_project_name(project_url)
    project_dir = os.path.join(root_dir, project_name)
    if os.path.exists(project_dir):
        install_cmd = (" ").join(base_args[:] + [
            f'cd {project_dir}',
            '&&', 
            'pip install --user --use-pep517 --upgrade -r requirements.txt'
        ])
        print(install_cmd)
        !$install_cmd

install_repo_env(
    project_url = 'https://github.com/bmaltais/kohya_ss.git',
    root_dir = root_dir,
    base_args = base_args
)

## 安装BLIP

In [ ]:
# !pip install blip blip-ci

In [ ]:
# !{" ".join(base_args + ['pip install fairscale==0.4.13 timm==0.6.12 requests==2.28.2','&&','pip install --index-url https://pypi.python.org/simple/ --upgrade blip'])}

# 重装accelerate

In [ ]:
# !pip uninstall accelerate -y
# !pip install accelerate

## 交代不同环境的挂载方式

In [ ]:
if env_name == 'AutoDL':
    copy_or_mount = False
elif env_name == 'OpenBayes':
    copy_or_mount = True

# 挂载基础模型

In [ ]:
import yaml

def read_yaml_file(file_path):
    with open(file_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    return yaml_data

# 调用函数并传入YAML文件路径
models_yaml_path = os.path.join(repo_root_dir,'./configs/models_list/models.yaml')
models_list = read_yaml_file(models_yaml_path)

In [ ]:
# Stable-diffusion 模型
sd_models = models_list['sd_models']

# 创建base_model目录
base_model_dir = os.path.join(root_dir, 'base_model')
if not os.path.exists(base_model_dir):
    os.mkdir(base_model_dir)

# 遍历模型列表并创建符号链接
for model in sd_models:
    check_model_and_make_symbollink(model, base_model_dir, copy_or_mount)

In [ ]:
# 创建base_model目录
base_model_dir = os.path.join(root_dir, 'base_model')
if not os.path.exists(base_model_dir):
    os.mkdir(base_model_dir)
check_model_and_make_symbollink('chilloutmix_NiPrunedFp16.safetensors',base_model_dir,copy_or_mount)

# 挂载BLIP模型

In [ ]:
# import os

# torch_hub_checkpoints_dir = os.path.abspath(os.path.join(root_dir,'.torch/hub/checkpoints/'))
# print(torch_hub_checkpoints_dir)
# # 该模型的下载地址为：https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
# check_model_and_make_symbollink('model_large_caption.pth',torch_hub_checkpoints_dir,copy_or_mount)

# check_model_and_make_symbollink('ViT-L-14_openai_artists.safetensors','./models/clip-interrogator',copy_or_mount)
# check_model_and_make_symbollink('ViT-L-14_openai_flavors.safetensors','./models/clip-interrogator',copy_or_mount)
# check_model_and_make_symbollink('ViT-L-14_openai_mediums.safetensors','./models/clip-interrogator',copy_or_mount)
# check_model_and_make_symbollink('ViT-L-14_openai_movements.safetensors','./models/clip-interrogator',copy_or_mount)
# check_model_and_make_symbollink('ViT-L-14_openai_negative.safetensors','./models/clip-interrogator',copy_or_mount)
# check_model_and_make_symbollink('ViT-L-14_openai_trendings.safetensors','./models/clip-interrogator',copy_or_mount)

## 创建python软连接以适应win版本代码

In [ ]:
# import os

# def check_symlink(source_path, symlink_path, symlink_name):
#     # 将软链接路径拼接成完整的文件路径
#     symlink_file = f"{symlink_path}/{symlink_name}"

#     # 如果软链接文件存在，则先删除它
#     if os.path.exists(symlink_file):
#         os.remove(symlink_file)

#     # 创建软链接
#     os.symlink(source_path, symlink_file)

#     # 验证软链接是否成功创建
#     if os.path.islink(symlink_file):
#         # print(f"{symlink_file} 是一个符号链接。")
#         print(f"\033[1m\033[92m{symlink_file}\033[0m ➜ \033[34m{os.readlink(symlink_file)}\033[0m")
#     else:
#         print(f"{symlink_file} 不是一个符号链接。")

# def create_env_and_link_python(project_dir, custom_env_name):
#     # 使用conda获取custom_env_name环境的Python路径
#     python_path = !conda run -n {custom_env_name} which python

#     # 创建venv目录
#     venv_dir = os.path.join(project_dir, '.venv')
#     if not os.path.exists(venv_dir):
#         os.mkdir(venv_dir)

#     # 创建软连接
#     link_path = os.path.join(venv_dir, 'Scripts', 'python.exe')
#     if not os.path.exists(os.path.join(venv_dir, 'Scripts')):
#         os.makedirs(os.path.join(venv_dir, 'Scripts'))
#     if not os.path.exists(link_path):
#         os.symlink(python_path[0], link_path)
#     # 检查软链接是否创建成功
#     check_symlink(python_path[0], os.path.join(venv_dir, 'Scripts'), 'python.exe')
        
# create_env_and_link_python(project_dir, custom_env_name)


## 批量临时修改kohya_ss的python可执行文件路径

In [ ]:
project_url = 'https://github.com/bmaltais/kohya_ss.git'
project_name = get_project_name(project_url)
project_dir = os.path.join(root_dir, project_name)

import os
import re

def try_open_file(file_path, encoding):
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            content = f.read()
        return content
    except UnicodeDecodeError:
        return None

def replace_python_path(project_dir, custom_env_name):
    # 获取custom_env_name环境的Python路径
    python_path = !conda run -n {custom_env_name} which python

    # 遍历目录，查找所有文件
    for root, dirs, files in os.walk(project_dir):
        for file in files:
            file_path = os.path.join(root, file)

            # 尝试打开文件，使用不同的编码方式
            content = try_open_file(file_path, 'utf-8')
            if content is None:
                content = try_open_file(file_path, 'gbk')

            # 如果仍然无法打开文件，则跳过该文件
            if content is None:
                continue

            # 查找并替换字符串
            old_str = '.\\venv\\Scripts\\python.exe'
            new_str = python_path[0]
            content_new = content.replace(old_str, new_str)

            old_str = '.\\\\venv\\\\Scripts\\\\python.exe'
            content_new = content_new.replace(old_str, new_str)

            # 如果文件内容没有被修改，则跳过该文件
            if content_new == content:
                continue

            # 保存文件
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(content_new)

            # 打印被修改的文件的路径信息
            print(f"文件 {file_path} 已被修改。")


replace_python_path(project_dir, custom_env_name)

# 重启tensorboard服务

In [ ]:
!pip install tensorboard --upgrade

In [ ]:
!kill $(lsof -t -i:6006)

# <i class="fa fa-rocket"></i> 启动Kohya_ss

import psutil

port = 8877

# 遍历所有正在运行的进程
for proc in psutil.process_iter():
    try:
        # 获取进程的端口信息
        connections = proc.connections()
        for conn in connections:
            if conn.status == psutil.CONN_LISTEN and conn.laddr.port == port:
                print(f"找到在端口 {port} 上运行的进程：{proc.pid}")
                # 杀掉进程
                proc.kill()
                print(f"已成功杀死进程 {proc.pid}")
                break
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

In [ ]:
project_url = 'https://github.com/bmaltais/kohya_ss.git'
project_name = get_project_name(project_url)
project_dir = os.path.join(root_dir, project_name)

%cd $project_dir

launch_args = base_args[:] + [
    f'cd {project_dir}','&&',
    # 'pip install --user "opencv-python-headless<4.3" ','&&',
    'python kohya_gui.py',
    '--server_port 8877',
]
cmd = (" ").join(launch_args)
print(launch_args)

!$cmd

In [ ]:
# !pip install --user --upgrade tensorflow opencv-python

In [ ]:
# !/usr/local/bin/python \
# "finetune/make_captions.py" \
# --batch_size="1" \
# --num_beams="1" \
# --top_p="0.9" \
# --max_length="75" \
# --min_length="5" \
# --beam_search \
# --caption_extension=".txt" \
# "/openbayes/home/training_sessions/LoRA/image" \
# --caption_weights="https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth"